# Imports

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel
from tqdm import tqdm

c:\Users\sushi\Documents\University of Toronto\MScAC\mat1510\subnetworks-in-rl\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Prompting

In [2]:
prompts = [
    'Solve for the unknown quantity in the equation $4 \phi + 5 = 6 \phi + 7$.', 
    'Solve the given system or show that no solution exists: $\begin{align*} x + 2y &= 1 \\ 3x +2y + 4z &= 7 \\ -2x + y - 2z &= -1 \end{align*}$.',
    'Two radar stations at points $A$ and $B$ are tracking a certain ship. Station $B$ is located 6 km east of station $A$. At a certain instant, the ship is 5 km from $A$ and also 5 km from $B$. Suppose the ship is to the north of $A$ and $B$. Find out where the ship is located.',
]

<>:2: SyntaxWarning: invalid escape sequence '\p'
<>:3: SyntaxWarning: invalid escape sequence '\e'
<>:2: SyntaxWarning: invalid escape sequence '\p'
<>:3: SyntaxWarning: invalid escape sequence '\e'
C:\Users\sushi\AppData\Local\Temp\ipykernel_25716\620494591.py:2: SyntaxWarning: invalid escape sequence '\p'
  'Solve for the unknown quantity in the equation $4 \phi + 5 = 6 \phi + 7$.',
C:\Users\sushi\AppData\Local\Temp\ipykernel_25716\620494591.py:3: SyntaxWarning: invalid escape sequence '\e'
  'Solve the given system or show that no solution exists: $\begin{align*} x + 2y &= 1 \\ 3x +2y + 4z &= 7 \\ -2x + y - 2z &= -1 \end{align*}$.',


In [ ]:
qwen2_5_rl = 'Qwen/Qwen2.5-Math-1.5B-Instruct'
rl_tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
models = [
    'Qwen/Qwen2.5-Math-1.5B'
    'zero_subnetwork', 
    'zero_non_subnetwork', 
    'random_subnetwork_seed42',
    'Qwen/Qwen2.5-Math-1.5B-Instruct'
    ]

In [5]:
def get_responses(models, prompts):
    responses = {}

    for name in tqdm(models):
        responses[name] = {'CoT': [], 'TIR': []}
        model = AutoModelForCausalLM.from_pretrained(name)
        model.eval()
        
        with torch.no_grad():
            for prompt in tqdm(prompts):
                cot_msg = [
                    {"role": "system", "content": "Please reason step by step, and put your final answer within \\boxed{}."},
                    {"role": "user", "content": prompt}
                    ]
                text = tokenizer.apply_chat_template(
                        cot_msg,
                        tokenize=False,
                        add_generation_prompt=True
                    )
                model_inputs = tokenizer([text], return_tensors="pt")

                generated_ids = model.generate(
                    **model_inputs,
                    max_length=1028
                )
                generated_ids = [
                    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
                ]

                responses[name]['CoT'].append(tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0])

                tir_msg = [
                    {"role": "system", "content": "Please integrate natural language reasoning with programs to solve the problem above, and put your final answer within \\boxed{}."},
                    {"role": "user", "content": prompt}
                ]
                text = tokenizer.apply_chat_template(
                        tir_msg,
                        tokenize=False,
                        add_generation_prompt=True
                    )
                model_inputs = tokenizer([text], return_tensors="pt")

                generated_ids = model.generate(
                    **model_inputs,
                    max_length=1028
                )
                generated_ids = [
                    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
                ]

                responses[name]['TIR'].append(tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0])
    return responses

In [7]:
responses = get_responses(model_names, prompts)

100%|██████████| 5/5 [3:01:51<00:00, 2182.31s/it]


In [9]:
from pprint import pprint
pprint(responses)

{'Qwen/Qwen2.5-Math-1.5B-Instruct': {'CoT': ['To solve the equation \\(4 \\phi '
                                             '+ 5 = 6 \\phi + 7\\) for the '
                                             'unknown quantity \\(\\phi\\), we '
                                             'will follow a step-by-step '
                                             'approach to isolate \\(\\phi\\) '
                                             'on one side of the equation.\n'
                                             '\n'
                                             '1. **Subtract \\(4 \\phi\\) from '
                                             'both sides of the equation:**\n'
                                             '   \\[\n'
                                             '   4 \\phi + 5 - 4 \\phi = 6 '
                                             '\\phi + 7 - 4 \\phi\n'
                                             '   \\]\n'
                                             '   Simplifying

In [10]:
import json

fp = 'responses_simpler_maxlength1028.json'

with open(fp, 'w', encoding='utf-8') as json_file:
    json.dump(responses, json_file, indent=4, ensure_ascii=False)